# multi-task_model-train

`multi-task_model-train.ipynb`

End of August attempts to create good model training workflows for multi-task experiments

Author: Jacob A Rose  
Created on: Monday August 29th, 2021

In [1]:
%load_ext autoreload
%autoreload 2

# Imports & definitions


In [2]:
from IPython.core.interactiveshell import InteractiveShell
# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"
import os
if 'TOY_DATA_DIR' not in os.environ: 
    os.environ['TOY_DATA_DIR'] = "/media/data_cifs/projects/prj_fossils/data/toy_data"
default_root_dir = os.environ['TOY_DATA_DIR']
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
import torch
from torch import nn
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
import timm
from rich import print as pp
import matplotlib.pyplot as plt
import pandas as pd
from munch import Munch
from lightning_hydra_classifiers.data.utils.make_catalogs import *
from lightning_hydra_classifiers.utils.metric_utils import get_per_class_metrics, get_scalar_metrics
from lightning_hydra_classifiers.utils.logging_utils import get_wandb_logger
import wandb
torch.manual_seed(17)

In [ ]:
from lightning_hydra_classifiers.scripts.train_multitask import PlantDataModule, LitMultTaskModule, ImagePredictionLogger, train_source_task, cmdline_args

## Datasets & DataModules

In [3]:
# from lightning_hydra_classifiers.experiments.transfer_experiment import TransferExperiment
# class PlantDataModule(pl.LightningDataModule):
# #     valid_tasks = (0, 1)
    
#     def __init__(self, 
#                  batch_size,
#                  task_id: int=0,
#                  image_size: int=224,
#                  image_buffer_size: int=32,
#                  num_workers: int=4,
#                  pin_memory: bool=True):
#         super().__init__()
#         self.batch_size = batch_size
#         self.num_workers = num_workers
#         self.pin_memory = pin_memory
        
        
#         self.experiment = TransferExperiment()
#         self.set_task(task_id)        
        
#         self.image_size = image_size
#         self.image_buffer_size = image_buffer_size
#         self.mean = [0.485, 0.456, 0.406]
#         self.std = [0.229, 0.224, 0.225]
#         # Train augmentation policy
        
#         self.__init_transforms()
                
#         self.tasks = self.experiment.get_multitask_datasets(train_transform=self.train_transform,
#                                                             val_transform=self.val_transform)


#     def __init_transforms(self):
        
#         self.train_transform = transforms.Compose([
#             transforms.RandomResizedCrop(size=self.image_size,
#                                          scale=(0.25, 1.2),
#                                          ratio=(0.7, 1.3),
#                                          interpolation=2),
#             torchvision.transforms.ToTensor(),
#             transforms.RandomHorizontalFlip(),
#             transforms.Normalize(self.mean, self.std),
#             transforms.Grayscale(num_output_channels=3)
#         ])

#         self.val_transform = transforms.Compose([
#             transforms.Resize(self.image_size+self.image_buffer_size),
#             torchvision.transforms.ToTensor(),
#             transforms.CenterCrop(self.image_size),
#             transforms.Normalize(self.mean, self.std),
#             transforms.Grayscale(num_output_channels=3)            
#         ])

#     def set_task(self, task_id: int):
#         assert task_id in self.experiment.valid_tasks
#         self.task_id = task_id
        
        
        
#     @property
#     def current_task(self):
#         return self.tasks[self.task_id]

#     def setup(self, stage=None):
#         task = self.current_task
#         # Assign train/val datasets for use in dataloaders
#         if stage == 'fit' or stage is None:
#             self.train_dataset = task['train']
#             self.val_dataset = task['val']
            
#             self.classes = self.train_dataset.classes
#             self.num_classes = len(self.train_dataset.label_encoder)
            
#         elif stage == 'test':
#             self.test_dataset = task['test']
                        
#     def train_dataloader(self):
#         return torch.utils.data.DataLoader(self.train_dataset,
#                           batch_size=self.batch_size,
#                           pin_memory=self.pin_memory,
#                           shuffle=True,
#                           num_workers=self.num_workers,
#                           drop_last=True)

#     def val_dataloader(self):
#         return torch.utils.data.DataLoader(self.val_dataset,
#                           batch_size=self.batch_size,
#                           pin_memory=self.pin_memory,
#                           num_workers=self.num_workers)
    
#     def test_dataloader(self):
#         return torch.utils.data.DataLoader(self.test_dataset,
#                           batch_size=self.batch_size,
#                           pin_memory=self.pin_memory,
#                           num_workers=self.num_workers)

## Model & LightningModules

In [4]:
# class CustomResNet(nn.Module):
#     def __init__(self,
#                  num_classes: int,
#                  model_name='resnet18',
#                  pretrained=False):
#         super().__init__()
#         self.num_classes = num_classes
#         self.model = timm.create_model(model_name, pretrained=pretrained)
#         self.in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(self.in_features, self.num_classes)

#     def forward(self, x):
#         x = self.model(x)
#         return x

# class LitMultiTaskModule(pl.LightningModule):
#     def __init__(self, config):
#         super().__init__()
#         self.config = config
#         self.lr = config['lr']
#         self.num_classes = config['num_classes']
# #         self.save_hyperparameters()
#         self._init_model(config)
#         self.metrics = self._init_metrics(stage='all')
# #         self.metric = pl.metrics.F1(num_classes=CONFIG['num_classes'])
#         self.criterion = nn.CrossEntropyLoss()
        

#     def forward(self, x, *args, **kwargs):
#         return self.model(x)

#     def configure_optimizers(self):
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
#         self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=self.config['t_max'], eta_min=self.config['min_lr'])

#         return {'optimizer': self.optimizer, 'lr_scheduler': self.scheduler}

#     def training_step(self, batch, batch_idx):
#         image = batch[0]
#         target = batch[1]
#         output = self.model(image)
#         loss = self.criterion(output, target)
# #         scores = self.metrics_train(output.argmax(1), target)
#         scores = self.metrics_train(output, target)
#         self.log_dict({"train_loss": loss, 'lr': self.optimizer.param_groups[0]['lr']},
#                       on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log_dict(scores,
#                       on_step=False, on_epoch=True, prog_bar=True, logger=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         image = batch[0]
#         target = batch[1]
#         output = self.model(image)
#         loss = self.criterion(output, target)
# #         scores = self.metrics_val(output.argmax(1), target)
#         scores = self.metrics_val(output, target)
        
#         self.log("val_loss", loss,
#                   on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log_dict(scores,
#                       on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    
    
#     def _init_model(self, config):
#         self.model =  CustomResNet(config["num_classes"],
#                                    model_name=config["model_name"],
#                                    pretrained=config["pretrained"])
    
#     def _init_metrics(self, stage: str='train'):
        
#         if stage in ['train', 'all']:
#             self.metrics_train = get_scalar_metrics(num_classes=self.num_classes, average='macro', prefix='train')
# #             self.metrics_train_per_class = get_per_class_metrics(num_classes=self.num_classes, prefix='train')
            
#         if stage in ['val', 'all']:
#             self.metrics_val = get_scalar_metrics(num_classes=self.num_classes, average='macro', prefix='val')
# #             self.metrics_val_per_class = get_per_class_metrics(num_classes=self.num_classes, prefix='val')
            
#         if stage in ['test', 'all']:
#             self.metrics_test = get_scalar_metrics(num_classes=self.num_classes, average='macro', prefix='test')
# #             self.metrics_test_per_class = get_per_class_metrics(num_classes=self.num_classes, prefix='test')

    

# Define & Run Experiment

## Config

In [5]:
#     "model":
#         {"backbone":{
#                  "name":'resnet50',
#                  "pretrained":True},
config = Munch({
    "seed":42,
    "model_name":'resnet50',
    "pretrained":True,
    "image_size": 224,
    "image_buffer_size": 32, 
    "num_classes": None,
    "lr": 5e-4,
    "min_lr": 1e-6,
    "t_max": 20,
    "num_epochs": 10,
    "batch_size": 32,
#     accum = 1,
    "precision": 16,
    "device": torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    "num_workers": 4,
    "pin_memory": True
})


# Seed everything
pl.seed_everything(config['seed'])

Global seed set to 42


42

In [6]:
import argparse


p = argparse.ArgumentParser(description="")
model_parser = p.add_parser('model', help='parser one')


# subparsers = p.add_subparsers()
# model_parser = subparsers.add_parser('model', help='parser one')
# subparser_one = p.add_subparsers()


print(p.parse_args(["-h"]))




In [3]:
import argparse
from munch import Munch
from rich import print as pp
import torch
import json


parser = argparse.ArgumentParser(description="")
subparsers = parser.add_subparsers()

p = subparsers.add_parser('', help='model args')
# subparser_one = parser_one.add_subparsers()
p.add_argument("-o", "--output_dir", dest="output_dir", type=str,
               default="/media/data_cifs/projects/prj_fossils/users/jacob/experiments/July2021-Nov2021/experiment_logs",
               help="Output root directory for experiment logs.")
p.add_argument("-e", "--num_epochs", dest="num_epochs", type=int, default=10,
               help="Number of training epochs")
p.add_argument("-res", "--image_size", dest="image_size", type=int, default=224,
               help="image_size/model input resolution")
p.add_argument("-buffer", "--image_buffer_size", dest="image_buffer_size", type=int, default=32,
               help="Additional resolution to resize images before either random (train) or center (test) crop to image_size.")
p.add_argument("-bsz", "--batch_size", dest="batch_size", type=int, default=32,
               help="batch_size")
p.add_argument("-nproc", "--num_workers", dest="num_workers", type=int, default=4,
               help="num_workers per dataloader")

p.add_argument("-model", "--model_name", dest="model_name", type=str, default="resnet50",
               help="model backbone architecture")
p.add_argument("-pre", "--pretrained", dest="pretrained", choices=["imagenet", True, False],
               help="Use pretrained imagenet weights or randomly initialize from scratch.")
p.add_argument("-lr", "--learning_rate", dest="lr", type=float, default=3e-4,
               help="Initial learning rate.")

args = parser.parse_args([""])

config = Munch({
    "seed":42,
    "num_epochs": args.num_epochs,
    "precision": 16,
    "device": torch.device('cuda' if torch.cuda.is_available() else 'cpu')
})

config.data = {
    "image_size": args.image_size,
    "image_buffer_size": args.image_buffer_size,
    "batch_size": args.batch_size,
    "num_workers": args.num_workers,
    "pin_memory": True
}
config.model = {"model_name": args.model_name,
                "pretrained": args.pretrained,
                "lr": args.lr,
                "num_classes": None,
                "t_max": 20,
                "min_lr": 1e-6}

config.callbacks = {"monitor": {"metric":"val_loss",
                                "mode": "min"}
                   }

out = json.dumps(config, default=str)
# pp(out)

In [14]:
import collections
# dir(collections)
argparse.Namespace()

argparse.Namespace

## DataModule

In [6]:
datamodule = PlantDataModule(batch_size=config.batch_size,
                             task_id=0,
                             image_size=config.image_size,
                             image_buffer_size=config.image_buffer_size,
                             num_workers=config.num_workers,
                             pin_memory=config.pin_memory)

datamodule.setup("fit")
config.num_classes = datamodule.num_classes

pp(config)
model = LitMultiTaskModule(config)

Munch({'seed': 42, 'model_name': 'resnet50', 'pretrained': True, 'image_size': 224, 
'image_buffer_size': 32, 'num_classes': 92, 'lr': 0.0005, 'min_lr': 1e-06, 't_max': 20, 
'num_epochs': 10, 'batch_size': 32, 'precision': 16, 'device': device(type='cuda'), 
'num_workers': 4, 'pin_memory': True})

## Callbacks

In [7]:
# Checkpoint
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=True,
                                      filename='checkpoint/{epoch:02d}-{val_loss:.4f}-{val_f1:.4f}',
                                      verbose=True,
                                      mode='min')
earlystopping = pl.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')

## Logger

In [8]:
wandb_logger = pl.loggers.WandbLogger(entity = "jrose",
                           project = "image_classification_train",
                           job_type = "train_supervised",
                           config=config,
                           group='ResNet')

## Trainer

In [9]:
# Initialize a trainer
trainer = pl.Trainer(
            limit_train_batches=0.1,
            limit_val_batches=0.1,
            max_epochs=config['num_epochs'],
            gpus=1,
#             accumulate_grad_batches=CONFIG['accum'],
            precision=config['precision'],
            callbacks=[earlystopping,
                       checkpoint_callback],
#                        ImagePredictionLogger(val_samples)],
#             checkpoint_callback=checkpoint_callback,
            logger=wandb_logger,
            weights_summary='top')

# datamodule.train_dataset[0]
type(datamodule.val_dataset)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


lightning_hydra_classifiers.data.utils.make_catalogs.CSVDataset

In [ ]:
from torchinfo import summary

# model_stats = summary(your_model, (1, 3, 28, 28), verbose=0)

# TRAIN

In [10]:
# Train the model ⚡🚅⚡
trainer.fit(model, datamodule)

# Close wandb run
wandb.finish() 

# datamodule.num_classes
# sorted(datamodule.classes)



In [2]:
import timm
from pprint import pprint
model_names = timm.list_models("effi*net*") #'*resne*t*')
pprint(model_names)

['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',
 'efficientnet_cc_b0_4e',
 'efficientnet_cc_b0_8e',
 'efficientnet_cc_b1_8e',
 'efficientnet_el',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_l2',
 'efficientnet_lite0',
 'efficientnet_lite1',
 'efficientnet_lite2',
 'efficientnet_lite3',
 'efficientnet_lite4']


In [15]:
import timm
from pprint import pprint
model_names = timm.list_models("senet*") #'*resne*t*')
pprint(model_names)

['senet154']


In [1]:
from lightning_hydra_classifiers.models.backbones.backbone import build_model

In [3]:
import inspect
dir(inspect)
print(inspect.getsource(build_model))

In [1]:
import os
from torchinfo import summary
from torch import nn
from typing import *
def log_model_summary(model: nn.Module,
                      working_dir: str,
                      input_size: Tuple[int],
                      full_summary: bool=True,
                      verbose: bool=1):
    """
    produce a text file with the model summary
    
    TODO: Add this to Eval Plugins
    
    log_model_summary(model=model,
                  working_dir=working_dir,
                  input_size=(1, data_config.channels, *data_config.image_size),
                  full_summary=True)

    """

    if full_summary:
        col_names=("kernel_size", "input_size","output_size", "num_params", "mult_adds")
    else:
        col_names=("input_size","output_size", "num_params")

    model_summary = summary(model.cuda(),
                            input_size=input_size,
                            row_settings=('depth', 'var_names'),
                            col_names=col_names,
                            verbose=verbose)

    model_summary_file = os.path.join(working_dir, 'model', f'{MODEL_NAME}_model_summary.txt')
    os.makedirs(os.path.dirname(model_summary_file), exist_ok=True)

    with open(model_summary_file, "w") as f:
        f.write(str(model_summary))
        
    return model_summary


In [2]:
from lightning_hydra_classifiers.models.backbones.backbone import build_model

# m_resnet = build_model("resnet50", pretrained=True)
# m_efficient = build_model("efficientnet_b2", pretrained=True)
# m_se_resnet = build_model("se_resnet152", pretrained=True)

model_names = ["resnet50", "efficientnet_b2", "se_resnet152"]
models = {}

for name in model_names:
    models[name] = build_model(name, pretrained=True)

# dir(model)

[BUILDING MODEL] build_model(resnet50, pretrained=True)
[BUILDING MODEL] build_model(efficientnet_b2, pretrained=True)
[BUILDING MODEL] build_model(se_resnet152, pretrained=True)


In [3]:
summary(models[name])

Layer (type:depth-idx)                        Param #
SENet                                         --
├─Sequential: 1-1                             --
│    └─Conv2d: 2-1                            9,408
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
├─Sequential: 1-2                             --
│    └─SEResNetBottleneck: 2-5                --
│    │    └─Conv2d: 3-1                       4,096
│    │    └─BatchNorm2d: 3-2                  128
│    │    └─Conv2d: 3-3                       36,864
│    │    └─BatchNorm2d: 3-4                  128
│    │    └─Conv2d: 3-5                       16,384
│    │    └─BatchNorm2d: 3-6                  512
│    │    └─ReLU: 3-7                         --
│    │    └─SEModule: 3-8                     8,464
│    │    └─Sequential: 3-9                   16,896
│    └─SEResNetBottleneck: 2-6                --
│    │    └─Conv2d: 3-10               

In [11]:
model.input_space

'RGB'

In [12]:
model.input_size

[3, 224, 224]

In [8]:
import timm
timm.__version__

'0.4.5'

In [6]:
model.feature_info

ModuleAttributeError: 'SENet' object has no attribute 'feature_info'

In [5]:
model = models["se_resnet152"]
working_dir = "./se_resnet152"

log_model_summary(model=model,
                  working_dir=working_dir,
                  input_size=(1, 3,224,224))

Layer (type (var_name):depth-idx)                            Kernel Shape              Input Shape               Output Shape              Param #                   Mult-Adds
SENet                                                        --                        --                        --                        --                        --
├─Sequential (layer0): 1-1                                   --                        [1, 3, 224, 224]          [1, 64, 56, 56]           --                        --
│    └─Conv2d (conv1): 2-1                                   [3, 64, 7, 7]             [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     118,013,952
│    └─BatchNorm2d (bn1): 2-2                                [64]                      [1, 64, 112, 112]         [1, 64, 112, 112]         128                       128
│    └─ReLU (relu1): 2-3                                     --                        [1, 64, 112, 112]         [1, 64, 112, 112]         --  

NameError: name 'os' is not defined

In [12]:
import timm
m = timm.create_model('efficientnet_b0', pretrained=True)
m.eval()

dir(m)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /media/data/jacob/pretrained_model_weights/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'act1',
 'act2',
 'add_module',
 'apply',
 'as_sequential',
 'bfloat16',
 'blocks',
 'bn1',
 'bn2',
 'buffers',
 '

In [16]:
import timm
m = timm.create_model('senet154', pretrained=True)
m.eval()

dir(m)

No pretrained weights exist for this model. Using random initialization.


['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'act1',
 'add_module',
 'apply',
 'bfloat16',
 'bn1',
 'buffers',
 'children',
 'conv1',
 'cpu',
 'cuda',
 'defaul

In [17]:
type(m)

timm.models.resnet.ResNet

In [8]:
m.feature_info

[{'stage': 1,
  'reduction': 2,
  'module': 'blocks.0.0',
  'hook_type': '',
  'num_chs': 16},
 {'stage': 2,
  'reduction': 4,
  'module': 'blocks.1.1',
  'hook_type': '',
  'num_chs': 24},
 {'stage': 3,
  'reduction': 8,
  'module': 'blocks.2.2',
  'hook_type': '',
  'num_chs': 40},
 {'stage': 5,
  'reduction': 16,
  'module': 'blocks.4.1',
  'hook_type': '',
  'num_chs': 112},
 {'stage': 7,
  'reduction': 32,
  'module': 'blocks.6.0',
  'hook_type': '',
  'num_chs': 960}]

In [7]:
# import timm
# m = timm.create_model('mobilenetv3_large_100', pretrained=True)
# m.eval()

m.default_cfg

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': (1, 1),
 'crop_pct': 0.875,
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'first_conv': 'conv_stem',
 'classifier': 'classifier'}

In [6]:
m.state_dict()

OrderedDict([('conv_stem.weight',
              tensor([[[[ 1.2824e-01,  1.4752e+00,  1.5283e+00],
                        [-1.7018e-01, -1.6418e+00, -1.5859e+00],
                        [ 7.6289e-02,  5.9333e-02,  1.4240e-01]],
              
                       [[ 2.8058e-01,  2.6544e+00,  2.4711e+00],
                        [-4.0636e-01, -2.8753e+00, -2.6655e+00],
                        [ 1.7505e-01,  1.6733e-01,  1.8481e-01]],
              
                       [[ 1.1134e-01,  9.2101e-01,  9.8720e-01],
                        [-1.2791e-01, -1.1501e+00, -1.0912e+00],
                        [ 9.3087e-02,  5.4900e-02,  2.1705e-01]]],
              
              
                      [[[-4.4489e-02,  5.4488e-02, -3.5371e-03],
                        [-2.4218e-01, -4.3948e-02,  4.8797e-02],
                        [-5.6930e-01, -7.6075e-02,  8.6943e-02]],
              
                       [[-1.1214e-01, -1.8766e-02, -7.5080e-02],
                        [-2.9495e-01, -2.

# Debugging

In [9]:
from lightning_hydra_classifiers.utils.common_utils import LabelEncoder
encoder = LabelEncoder()
experiment = TransferExperiment()

task_0 = experiment.setup_task_0()

task_0['train'].label_encoder

In [10]:
task_0['val'].label_encoder

<LabelEncoder>:
    num_classes: 93
    fit on num_samples: 16605

In [11]:
task_0['test'].label_encoder

<LabelEncoder>:
    num_classes: 20
    fit on num_samples: 2797

In [48]:
replace_class_indices = {"Nothofagaceae":"Fagaceae"}
task_0_label_encoder = task_0['train'].label_encoder
print(task_0_label_encoder)

task_0_label_encoder.__init__(replace = replace_class_indices)

print(task_0_label_encoder)

<LabelEncoder(num_classes=0)>
<num_replaced_classes=1>
<LabelEncoder(num_classes=0)>
<num_replaced_classes=1>


In [35]:
print(len(task_0_label_encoder.classes))
print(task_0_label_encoder)
task_0_label_encoder.fit(task_0['test'].targets)
print(len(task_0_label_encoder.classes))
print(task_0_label_encoder)
task_0_label_encoder.fit(task_0['train'].targets)
print(len(task_0_label_encoder.classes))
print(task_0_label_encoder)

In [58]:
print(task_0_label_encoder)



import collections

self = task_0_label_encoder
y = task_0['test'].targets

counts = collections.Counter(y)
print(self.num_samples)
self.num_samples += sum(counts.values())
print(self.num_samples)

classes = sorted(counts.keys())
print(classes)
print(len(classes))

In [53]:
# old_num_classes = len(self)
# print(f"old_num_classes={old_num_classes}")
# new_classes = sorted([label for label in classes if label not in self.classes])
# print(f'new_classes={new_classes}')
# print(f'len(new_classes)={len(new_classes)}')

old_num_classes = len(self)
print(f"old_num_classes={old_num_classes}")
new_classes = []
for label in classes:
    if (label not in self.classes) and (label not in self.replace):
        new_classes.append(label)
# new_classes = sorted([label for label in classes if label not in self.classes])
print(f'new_classes={new_classes}')
print(f'len(new_classes)={len(new_classes)}')

In [61]:
for i, label in enumerate(new_classes):
    self.class2idx[label] = old_num_classes + i
print(self.class2idx)
print(len(self.class2idx))

self.index2class = {v: k for k, v in self.class2idx.items()}

all_classes = []
for label in self.class2idx.keys():
    if label not in self.replace.keys():
        all_classes.append(label)
print(f"all_classes={all_classes}")
print(f"len(all_classes)={len(all_classes)}")

self.classes = sorted(all_classes)

In [64]:
# self.classes = [k for k in self.class2idx.keys() if k not in self.replace.keys()] 
print(len(self.class2idx), len(self.index2class), len(self.classes))

self.replace_class2idx_items()
print(len(self.class2idx), len(self.index2class), len(self.classes))

new_classes = [c for c in new_classes if c not in self.replace.keys()]
if len(new_classes):
    log.debug(f"[FITTING] {len(y)} samples with {len(classes)} classes, adding {len(new_classes)} new class labels. Latest num_classes = {len(self)}")
assert len(self) == (old_num_classes + len(new_classes))

In [ ]:
self.index2class = {v: k for k, v in self.class2idx.items()}

# all_classes = []

self.classes = [k for k in self.class2idx.keys() if k not in self.replace.keys()]        
self.replace_class2idx_items()

new_classes = [c for c in new_classes if c not in self.replace.keys()]
if len(new_classes):
    log.debug(f"[FITTING] {len(y)} samples with {len(classes)} classes, adding {len(new_classes)} new class labels. Latest num_classes = {len(self)}")
assert len(self) == (old_num_classes + len(new_classes))

(task_0['test'].label_encoder)

(task_0['test'].classes)



sorted(set(datamodule.train_dataset.targets))

len(sorted(set(datamodule.train_dataset.classes)))


len(sorted(set(datamodule.train_dataset.classes)))


datamodule.num_classes

import numpy as np
set(np.arange(len(datamodule.train_dataset.classes))) - set(datamodule.train_dataset.targets)

import numpy as np
set(datamodule.train_dataset.targets) - set(np.arange(len(datamodule.train_dataset.classes)))


len(sorted(set(datamodule.train_dataset.targets)))

model.metrics_train

model.metrics_val

## Export experiment

In [1]:
encoder_path = "/media/data/jacob/GitHub/lightning-hydra-classifiers/notebooks/experiments_August_2021/Extant-to-PNAS-512-transfer_benchmark/task_0/test.json"

from lightning_hydra_classifiers.utils.common_utils import LabelEncoder




In [2]:
encoder = LabelEncoder.load(encoder_path)

encoder


<LabelEncoder>:
    num_classes: 177
    fit on num_samples: 0

In [4]:
len(encoder.class2idx)

len(encoder.index2class)

177

In [5]:
set(encoder.class2idx.keys()) - set(encoder.index2class.values())

{'Fagaceae', 'Phyllanthaceae'}

```
LabelEncoder:  
    ::class2idx  
        - Needs to have a many-to-one mapping from text to int. All int labels are unique, but multiple class names can map to 1 int label.
    ::idx2class  
        - Needs to have a one-to-one mapping from int to text. For ints that map to more than 1 text label, this maps the int only to the correct standardized label for the experiment. As in, it maps only to the label we used to replace another label. e.g. If replacements includes {"NothoFagaceae": "Fagaceae"}, and both  of them maps to int label 16, then encoder.idx2class[16] must return "Fagaceae".
    ::classes  
    ::num_classes  
        - Needs to correspond to actual neural net output size, therefore excludes replaced classes
    ::replacements  
```

* len(idx2class) <= len(class2idx)
* num_classes == len(idx2class) <= len(class2idx)



1. Initialize blank label encoder
    - Provide replacements dict to allow backwards compatible mappings
        - e.g. Nothofagaceae (newer, Extant) -> Fagaceae (older, PNAS)

2. Fit encoder on $|y_s|$ to have 

In [1]:
from sklearn.datasets import make_classification

In [8]:
# x, y = make_classification(n_features=6, n_redundant=2, n_informative=4)
import random


# y = 

In [13]:
import numpy as np



loss = np.random.random(15)

sorted_idx = np.argsort(loss)

sorted_loss

array([13, 12,  2, 14, 11,  0,  7,  9,  8, 10,  5,  3,  6,  1,  4])

In [14]:
loss[sorted_loss]

array([0.00322549, 0.02245847, 0.06010219, 0.12793732, 0.13489323,
       0.19017105, 0.22378192, 0.27114762, 0.3105012 , 0.31955682,
       0.37527135, 0.69631603, 0.78453079, 0.82577511, 0.87675758])

In [1]:
from lightning_hydra_classifiers.experiments.transfer_experiment import TransferExperiment
output_root_dir = "/media/data_cifs/projects/prj_fossils/users/jacob/experiments/July2021-Nov2021/csv_datasets/experimental_datasets"
experiment = TransferExperiment()
experiment.export_experiment_spec(output_root_dir=output_root_dir)

Exporting experiment to experiment_dir: /media/data_cifs/projects/prj_fossils/users/jacob/experiments/July2021-Nov2021/csv_datasets/experimental_datasets/Extant-to-PNAS-512-transfer_benchmark
train 16605
val 4152
train 2011
val 503
19
92


In [6]:
self = experiment

replace_class_indices = {"Nothofagaceae":"Fagaceae"}

task_0 = self.setup_task_0()
task_1 = self.setup_task_1()

#         import pdb;pdb.set_trace()



#         print(f"__init__: {task_0['train'].label_encoder}")
task_0_label_encoder = task_0['train'].label_encoder
task_0_label_encoder.__init__(replacements = replace_class_indices)

# print(max(task_0_label_encoder.class2idx.values()))
task_0_label_encoder.class2idx

print(len(set(task_0['test'].targets)))





print(len(set(task_0_label_encoder.classes)))
task_0_label_encoder.fit(task_0['test'].targets)
print(len(set(task_0_label_encoder.classes)))

In [7]:
print(max(task_0_label_encoder.class2idx.values()))
task_0_label_encoder.class2idx

18


{'Anacardiaceae': 0,
 'Annonaceae': 1,
 'Apocynaceae': 2,
 'Betulaceae': 3,
 'Celastraceae': 4,
 'Combretaceae': 5,
 'Ericaceae': 6,
 'Fabaceae': 7,
 'Fagaceae': 8,
 'Lauraceae': 9,
 'Malvaceae': 10,
 'Melastomataceae': 11,
 'Myrtaceae': 12,
 'Passifloraceae': 13,
 'Phyllanthaceae': 14,
 'Rosaceae': 15,
 'Rubiaceae': 16,
 'Salicaceae': 17,
 'Sapindaceae': 18,
 'Nothofagaceae': 8}

In [8]:
print(len(set(task_0['train'].targets)))

93


In [ ]:


print(len(set(task_0_label_encoder.classes)))
task_0_label_encoder.fit(task_0['train'].targets)
print(len(set(task_0_label_encoder.classes)))

In [10]:


print(max(task_0_label_encoder.class2idx.values()))
task_0_label_encoder.class2idx

In [11]:
task_0_label_encoder.classes

task_0_label_encoder.idx2class

In [3]:
!rm -r "/media/data_cifs/projects/prj_fossils/users/jacob/experiments/July2021-Nov2021/csv_datasets/experimental_datasets/task_1"

In [2]:
class self:
    idx2class = {0:"test",
                 1:"2"}



old_highest_class = max(self.idx2class.keys())

In [3]:
old_highest_class

1

# Refactor LabelEncoder

In [13]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import numbers
from typing import Union, List, Any, Tuple, Dict, Optional, Sequence
import collections
from sklearn.model_selection import train_test_split
import json
from lightning_hydra_classifiers.utils import template_utils
from lightning_hydra_classifiers.utils.plot_utils import colorbar


log = template_utils.get_logger(__name__)


# __all__ = ["LabelEncoder", "trainval_split", "trainvaltest_split", "plot_split_distributions", "plot_class_distributions",
#            "filter_df_by_threshold", "compute_class_counts"]



class LabelEncoder(object):
    
    """Label encoder for tag labels."""
    def __init__(self,
                 class2idx: Dict[str,int]=None,
                 replacements: Optional[Dict[str,str]]=None):
        self.class2idx = class2idx or {}
        self.replacements = replacements or {}
#         self.idx2class = {v: k for k, v in self.class2idx.items() if k not in self.replacements.keys()}
#         self.classes = [k for k in self.class2idx.keys() if k not in self.replacements.keys()]
        
        assert len(self.classes) == len(self.idx2class) <= len(self.class2idx)
        self.num_samples = 0
        self.verbose=False
        self.replace_class2idx_items()
        

    @property
    def idx2class(self):
        return {v: k for k, v in self.class2idx.items() if k not in self.replacements.keys()}
    
    @property
    def classes(self):
        return [k for k in self.class2idx.keys() if k not in self.replacements.keys()]
    

        
    def replace_class2idx_items(self):
        """
        Update inplace self.class2idx mappings, so that any class labels in self.replacements.keys()
        map to the same int label as their corresponding value in self.replacements.values().
        
        """
        if (len(self.replacements) == 0) \
        or (len([k for k in self.replacements.keys() if k in self.class2idx.keys()]) == 0):
            # No-op if replacements keys are empty or have zero overlap with class2idx keys.
            return
        
        if self.verbose:
            log.info(f'LabelEncoder replacing {len(self.replacements.keys())} class encodings with that other an another class')
            log.info('Replacing: ' + str({k:v for k,v in self.replacements.items() if k in self.class2idx}))
        for old, new in self.replacements.items():
            if old in list(self.class2idx.keys()):
                self.class2idx[old] = self.class2idx[new]
#         self.idx2class = {v: k for k, v in self.class2idx.items()}
#         self.classes = [k for k in self.class2idx.keys() if k not in self.replacements.keys()]                
        
    def __len__(self):
        return len(self.idx2class)
#         return len(self.classes)

    def num_classes(self):
        return len(self)


    def __str__(self):
        msg = f"<LabelEncoder(num_classes={len(self)})>"
        if len(self.replacements) > 0:
            msg += "\n" + f"<num_replaced_classes={len(self.replacements)}>"
        return msg

    def fit(self, y):
        
        counts = collections.Counter(y)
        self.num_samples += sum(counts.values())
        
        classes = sorted(list(counts.keys()))
        new_classes = sorted([label for label in classes if label not in self.classes])
        
        old_num_classes = len(self)
        old_highest_class = max(self.idx2class.keys())
        for i, label in enumerate(new_classes):
            self.class2idx[label] = old_highest_class + i
#         self.idx2class = {v: k for k, v in self.class2idx.items()}
#         self.classes = [k for k in self.class2idx.keys() if k not in self.replacements.keys()]        
        self.replace_class2idx_items()

        new_classes = [c for c in new_classes if c not in self.replacements.keys()]
        if len(new_classes):
            log.debug(f"[FITTING] {len(y)} samples with {len(classes)} classes, adding {len(new_classes)} new class labels. Latest num_classes = {len(self)}")
        assert len(self) == (old_num_classes + len(new_classes))
        assert np.all([label in self.idx2class.values() for label in new_classes])
        return self

    def encode(self, y):
        if not hasattr(y,"__len__"):
            y = [y]
#         print(self.class2idx)
        return np.array([self.class2idx[label] for label in y])

    def decode(self, y):
        if not hasattr(y,"__len__"):
            y = [y]
        return np.array([self.idx2class[label] for label in y])

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = self.getstate() # {"class2idx": self.class2idx}
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)
    
    def getstate(self):
        return {"class2idx": self.class2idx,
                "replacements": self.replacements}
    
    def __repr__(self):
        disp = f"""<{str(type(self)).strip("'>").split('.')[-1]}>:\n"""
        disp += f"    num_classes: {len(self)}\n"
        disp += f"    fit on num_samples: {self.num_samples}"
        return disp

In [43]:


from lightning_hydra_classifiers.utils.common_utils import LabelEncoder
encoder = LabelEncoder()
experiment = TransferExperiment()
task_0 = experiment.setup_task_0()

old_encoder = task_0['train'].label_encoder

encoder = LabelEncoder()

print(f"Old:", old_encoder)
print("New, initialized:", encoder)

data = task_0['train']
data_df = data.samples_df

experiment

replace_class_indices = {"Nothofagaceae":"Fagaceae"}

(data.label_encoder.replace)

data_df

data_df = data_df.rename(columns={"family":"newest_family"})
data_df = data_df.assign(family = data_df.newest_family.replace(replace_class_indices))
data_df


# data_df = data_df.sort_values("family").astype({"family":pd.CategoricalDtype(),
#                           "newest_family":pd.CategoricalDtype()}) #family.cat #!=data_df.newest_family]

data_df = data_df.sort_values("family").astype({"family":pd.CategoricalDtype()})
data_df = data_df.convert_dtypes()
data_df.info()

dir(data_df.family[~data_df.family.duplicated(keep='first')])
# data_df.family.duplicated(keep='first')

y_col = "family"
category_df = data_df[y_col][~data_df[y_col].duplicated(keep='first')]   #.to_list()
class2index = dict(zip(category_df.to_list(), category_df.cat.codes.to_list()))
# data_df.family.duplicated(keep='first')

class2index

data_df.info()

dir(data_df.family.cat)

In [ ]:
len(set(data_df.family.cat.codes))

data_df.family.cat.ordered

data_df.family.cat.codes

data_df[data_df.family!=data_df.newest_family]

In [2]:
import numpy as np

np.transpose

<function numpy.transpose(a, axes=None)>

# BYOL

## Load and preprocess pre-formatted csv datasets and create train val test splits

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

from torchvision.models import mobilenet_v2, resnet50
from torchvision.datasets import STL10
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import os

import torch.nn as nn
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from lightning_hydra_classifiers.train_BYOL import *
torch.backends.cudnn.benchmark = True
from munch import Munch

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

if 'TOY_DATA_DIR' not in os.environ: 
    print(f"Setting env variable $TOY_DATA_DIR={os.environ['TOY_DATA_DIR']}")
    os.environ['TOY_DATA_DIR'] = "/media/data_cifs/projects/prj_fossils/data/toy_data"
default_root_dir = os.environ['TOY_DATA_DIR']
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"



config = Munch({"dataset_name":"Extant-PNAS",
                "model":{
                    "backbone":"resnet50"}
               })
# config = Munch({"dataset_name":"STL10"})
print(f"config: {config}")


transform = transforms.Compose([ToTensor(),
                               normalize])



if config.dataset_name == "STL10":
    from torchbearer.cv_utils import DatasetValidationSplitter

    train_data = STL10(os.environ['TOY_DATA_DIR'], split='train', transform=transform, download=True)
    test_data = STL10(os.environ['TOY_DATA_DIR'], split='test', transform=transform, download=True)
    
    splitter = DatasetValidationSplitter(len(train_data), 0.1)
    train_set = splitter.get_train_dataset(train_data)
    val_set = splitter.get_val_dataset(train_data)
    
else:
    exp = TransferExperiment()
    task_0, task_1 = exp.get_multitask_datasets(train_transform=transform,
                                                val_transform=transform)
    train_data, val_data, test_data = task_0["train"], task_0["val"], task_0["test"]
    train_set, val_set = train_data, val_data

classes = train_data.classes
num_classes = len(classes)
print('\n List of all classes: ')
print(classes)
print(f"len(classes)={len(classes)}")


BATCH_SIZE = 16
num_workers = 2
pin_memory = False

train_gen = torch.utils.data.DataLoader(train_set, pin_memory=pin_memory, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_gen = torch.utils.data.DataLoader(val_set, pin_memory=pin_memory, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)
test_gen = torch.utils.data.DataLoader(test_data, pin_memory=pin_memory, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

config: Munch({'dataset_name': 'Extant-PNAS', 'model': {'backbone': 'resnet50'}})

 List of all classes: 
['Fagaceae', 'Ericaceae', 'Fabaceae', 'Anacardiaceae', 'Rosaceae', 'Betulaceae', 'Salicaceae', 'Sapindaceae', 'Lauraceae', 'Rubiaceae', 'Celastraceae', 'Malvaceae', 'Myrtaceae', 'Apocynaceae', 'Melastomataceae', 'Passifloraceae', 'Combretaceae', 'Annonaceae', 'Phyllanthaceae', 'Clusiaceae', 'Sapotaceae', 'Lythraceae', 'Burseraceae', 'Bignoniaceae', 'Meliaceae', 'Malpighiaceae', 'Cunoniaceae', 'Onagraceae', 'Marantaceae', 'Santalaceae', 'Berberidaceae', 'Ochnaceae', 'Ebenaceae', 'Cornaceae', 'Sabiaceae', 'Schisandraceae', 'Araliaceae', 'Staphyleaceae', 'Hamamelidaceae', 'Dipterocarpaceae', 'Dichapetalaceae', 'Dilleniaceae', 'Proteaceae', 'Connaraceae', 'Caprifoliaceae', 'Piperaceae', 'Bonnetiaceae', 'Juglandaceae', 'Geraniaceae', 'Aquifoliaceae', 'Moraceae', 'Rutaceae', 'Cercidiphyllaceae', 'Orchidaceae', 'Orobanchaceae', 'Apiaceae', 'Violaceae', 'Altingiaceae', 'Eucommiaceae', 'Gne

In [4]:
# cm = PyCM().on_val().to_html_file('cm.{epoch}')

# We copy the final layer form MobileNetV2 and replace the linear layer with one to 10 channels

if config.model.backbone == "mobilenet_v2":
    model = mobilenet_v2(pretrained=True, progress=False)
    model.classifier = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(model.last_channel, num_classes),
            )

elif config.model.backbone == "resnet50":
    model = resnet50(pretrained=True, progress=False)
    model.fc = nn.Sequential(
                nn.Dropout(0.2),
                nn.Linear(model.fc.in_features, num_classes+1),
            )

In [11]:
# import torchsummary
# import torchinfo
import torch.optim as optim
# import torchbearer
# from torchbearer import Trial
# from torchbearer.callbacks import PyCM

# for k, m in model.named_modules():
#     if (k.startswith("features")) or (k.startswith("layer")):
#         print(f"Freezing: {k}")
#         m.requires_grad = False
#     else:
#         print(f"Unfreezing: {k}")
#         m.requires_grad = True
        
#     print(f"{k} : {m.requires_grad}")

freeze_at = "fc"

freeze_current=False
for k, m in model.named_modules():
    if (k == freeze_at) or freeze_current:
        freeze_current=True
        print(f"Unfreezing: {k}")
        m.requires_grad = True
    else:
        print(f"Freezing: {k}")
        m.requires_grad = False
        
    print(f"{k} : {m.requires_grad}")

In [ ]:
# cm = PyCM().on_val().to_pyplot(normalize=True, title='Confusion Matrix: {epoch}')
# cm_csv = PyCM().on_val().to_csv_file("cm_{epoch}")
# model = SimpleModel()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss = nn.CrossEntropyLoss()

# trial = Trial(model, optimizer, loss, metrics=['acc', 'loss'], callbacks=[cm,cm_csv]).to(device)
# trial.with_generators(train_generator=train_gen, val_generator=val_gen)
# history = trial.run(epochs=2, verbose=2)

In [ ]:
model = SelfSupervisedLearner(
    resnet,
    image_size = IMAGE_SIZE,
    hidden_layer = 'avgpool',
    projection_size = 256,
    projection_hidden_size = 4096,
    moving_average_decay = 0.99
)

trainer = pl.Trainer(
    gpus = NUM_GPUS,
    max_epochs = EPOCHS,
    accumulate_grad_batches = 1,
    sync_batchnorm = True
)

trainer.fit(model, train_loader)

In [3]:
import torchdata

# class UnsupervisedDatasetWrapper(torchdata.datasets.Files):
class UnsupervisedDatasetWrapper(torchvision.datasets.ImageFolder):
    
    def __init__(self, dataset):
        
        self.dataset = dataset
        
    def __getitem__(self, index):
        return self.dataset[index][0]
    
    def __len__(self):
        return len(self.dataset)
    
    def __repr__(self):
        out = "<UnsupervisedDatasetWrapper>\n"
        out += self.dataset.__repr__()
        return out

In [8]:
task_0['test'][0]

for subset in ["train","val","test"]:
    task_0[subset] = UnsupervisedDatasetWrapper(task_0[subset])
    task_1[subset] = UnsupervisedDatasetWrapper(task_1[subset])

task_0['test'][0]

type(task_0['test'].dataset)

task_0['test']#.dataset

### Create transforms

In [14]:
from torchvision import transforms
from typing import *


totensor: Callable = torchvision.transforms.ToTensor()

def toPIL(img: torch.Tensor, mode="RGB") -> Callable:
    return torchvision.transforms.ToPILImage(mode)


def normalize_transform(mean = [0.485, 0.456, 0.406],
                        std = [0.229, 0.224, 0.225]) -> Callable:
    return transforms.Normalize(mean=mean,
                                std=std)

def default_train_transforms(image_size: int=224,
                             normalize: bool=True, 
                             augment:bool=True,
                             grayscale: bool=True,
                             channels: Optional[int]=3,
                             mean = [0.485, 0.456, 0.406],
                             std = [0.229, 0.224, 0.225]):
    """Subclasses can override this or user can provide custom transforms at runtime"""
    transform_list = []
#         transform_jit_list = []
    resize_PIL = not augment
    if augment:
        transform_list.extend([transforms.RandomResizedCrop(size=image_size,
                                                            scale=(0.25, 1.2),
                                                            ratio=(0.7, 1.3),
                                                            interpolation=2),
                               totensor
                             ])
    return default_eval_transforms(image_size=image_size,
                                        normalize=normalize,
                                        resize_PIL=resize_PIL,
                                        grayscale=grayscale,
                                        channels=channels,
                                        transform_list=transform_list,
                                        mean=mean,
                                        std=std)

def default_eval_transforms(image_size: int=224,
                            image_buffer_size: int=32,
                            normalize: bool=True,
                            resize_PIL: bool=True,
                            grayscale: bool=True,
                            channels: Optional[int]=3,
                            transform_list: Optional[List[Callable]]=None,
                            mean = [0.485, 0.456, 0.406],
                            std = [0.229, 0.224, 0.225]):
    """Subclasses can override this or user can provide custom transforms at runtime"""
    transform_list = transform_list or []
    transform_jit_list = []

    if resize_PIL:
        # if True, assumes input images are PIL.Images (But need to check if this even matters.)
        # if False, expects input images to already be torch.Tensors
        transform_list.extend([transforms.Resize(image_size+image_buffer_size),
                               transforms.CenterCrop(image_size),
                               totensor])
    if normalize:
        transform_jit_list.append(normalize_transform(mean, std))

    if grayscale:
        transform_jit_list.append(transforms.Grayscale(num_output_channels=channels))

    return transforms.Compose([*transform_list, *transform_jit_list])


def get_default_transforms(image_size: int=224,
                           normalize: bool=True,
                           augment:bool=True,
                           grayscale: bool=True,
                           channels: Optional[int]=3,
                           mean = [0.485, 0.456, 0.406],
                           std = [0.229, 0.224, 0.225]):

    
    train_transform = default_train_transforms(image_size=image_size,
                                               normalize=normalize,
                                               augment=augment,
                                               grayscale=grayscale,
                                               channels=channels,
                                               mean=mean,
                                               std=std)
    eval_transform = default_eval_transforms(image_size=image_size,
                                             image_buffer_size=32,
                                             normalize=normalize,
                                             resize_PIL=not augment,
                                             grayscale=grayscale,
                                             channels=channels,
                                             transform_list=None,
                                             mean=mean,
                                             std=std)
    
    
    
    return train_transform, eval_transform

In [15]:
train_transform, val_transform = get_default_transforms(image_size=224,
                                                         normalize=True,
                                                         augment=True,
                                                         grayscale=True,
                                                         channels=3,
                                                         mean = [0.485, 0.456, 0.406],
                                                         std = [0.229, 0.224, 0.225])